Sebastian Yepes Acevedo

Cc:1007448816

In [1]:
import numpy as np

def compute_cost(AL, Y, eps=1e-12):
    """
    Coste de entropía cruzada binaria (vectorizado).

    AL: salida de la última capa, shape (1, m)
    Y : etiquetas verdaderas, shape (1, m)
    eps: pequeño valor para estabilidad numérica

    Returns:
      cost: float escalar
    """
    m = Y.shape[1]

    AL_clipped = np.clip(AL, eps, 1 - eps)

    # coste
    cost = - (1.0 / m) * np.sum(Y * np.log(AL_clipped) + (1 - Y) * np.log(1 - AL_clipped))

    return float(cost)


def compute_cost_and_dAL(AL, Y, eps=1e-12):
    """
    Devuelve coste y dA_L = dJ/dAL, útil para iniciar backprop.
    """
    m = Y.shape[1]
    AL_clipped = np.clip(AL, eps, 1 - eps)

    cost = - (1.0 / m) * np.sum(Y * np.log(AL_clipped) + (1 - Y) * np.log(1 - AL_clipped))

    # derivada de la función de coste respecto a AL
    # dJ/dAL = - (Y/AL) + ((1-Y)/(1-AL))
    dAL = - (np.divide(Y, AL_clipped) - np.divide(1 - Y, 1 - AL_clipped)) / m * m
    # nota: el /m y *m se cancelan; se deja así para claridad.
    # mejor simplificar:
    dAL = - (np.divide(Y, AL_clipped) - np.divide(1 - Y, 1 - AL_clipped))

    return float(cost), dAL

Se crea una clase de red neuronal, donde se está manejando de entrada la salida de la red neuronal en la utima capa, las etiquetas y el control de errores.

Una vez tenemos la base, se crea una función que calcula la función de coste a partir de la ecuación

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

Lueo se obtiene la derivada de la perdida respecto a la salida final $$dA_L=\frac{\partial J}{\partial A^{[L]}}$$


In [3]:
def relu(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def tanh(Z):
    return np.tanh(Z)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def sigmoid_backward(dA, Z):
    s = 1 / (1 + np.exp(-Z))
    return dA * s * (1 - s)

def tanh_backward(dA, Z):
    t = np.tanh(Z)
    return dA * (1 - t**2)


activation_backward = {
    relu: relu_backward,
    sigmoid: sigmoid_backward,
    tanh: tanh_backward
}


class NeuralNetwork:
    def __init__(self, topology, activations):
        """
        topology   : lista con la cantidad de neuronas por capa [n0, n1, ..., nL]
        activations: lista de activaciones [None, act1, act2, ..., actL]
        """
        self.topology = topology
        self.activations = activations
        self.L = len(topology) - 1  # número de capas no-iniciales

        # Inicializamos los parámetros
        self.W = {}
        self.b = {}

        self.Z_cache = {}
        self.A_cache = {}

        for l in range(1, len(topology)):
            n_l = topology[l]
            n_prev = topology[l-1]

            # Inicialización aleatoria
            self.W[l] = np.random.randn(n_l, n_prev) * 0.01
            self.b[l] = np.random.randn(n_l, 1) * 0.01

    def output(self, X):
        """
        Realiza forward propagation.
        X: matriz de entrada de tamaño (n[0], m)

        Returns:
            Z_list: lista de Z[l]
            A_list: lista de A[l] (siendo A[0] = X)
        """
        A = X
        A_list = [A]  # A[0]
        Z_list = [None]  # Z[0] no existe

        for l in range(1, self.L + 1):
            Z = self.W[l] @ A + self.b[l]
            A = self.activations[l](Z) if self.activations[l] is not None else Z

            Z_list.append(Z)
            A_list.append(A)

        return Z_list, A_list
    def forward(self, A0):
        """
        Forward pass interno.
        Guarda Z[l] y A[l] en cache.
        """
        self.A_cache[0] = A0
        A = A0

        for l in range(1, self.L + 1):
            Z = self.W[l] @ A + self.b[l]
            A = self.activations[l](Z) if self.activations[l] else Z

            self.Z_cache[l] = Z
            self.A_cache[l] = A

        return A


    def backward(self, Y):
        """
        Realiza backward propagation completo.

        Y: etiquetas reales (1, m)

        Returns:
            grads: diccionario con dW[l], db[l], dA[l]
        """
        grads = {}
        m = Y.shape[1]

        # Recuperamos la salida final A[L]
        AL = self.A_cache[self.L]

        # Paso 1: dAL (derivada de la pérdida)
        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

        # Paso 2: capa L
        ZL = self.Z_cache[self.L]
        act_L = self.activations[self.L]

        dZL = activation_backward[act_L](dAL, ZL)
        dWL = (1/m) * (dZL @ self.A_cache[self.L - 1].T)
        dbL = (1/m) * np.sum(dZL, axis=1, keepdims=True)
        dA_prev = self.W[self.L].T @ dZL

        grads[f"dW{self.L}"] = dWL
        grads[f"db{self.L}"] = dbL

        # ---- Paso 3: capas L-1 ... 1
        for l in reversed(range(1, self.L)):
            Z = self.Z_cache[l]
            A_prev = self.A_cache[l-1]
            act = self.activations[l]

            dZ = activation_backward[act](dA_prev, Z)
            dW = (1/m) * (dZ @ A_prev.T)
            db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            dA_prev = self.W[l].T @ dZ

            grads[f"dW{l}"] = dW
            grads[f"db{l}"] = db

        return grads

    def compute_cost(self, Y, eps=1e-12):
        """
        Calcula el coste usando la salida guardada en A_cache[L].
        Y: etiquetas shape (1, m)
        """
        AL = self.A_cache[self.L]  # salida final
        return compute_cost(AL, Y, eps)

    def compute_cost_and_dAL(self, Y, eps=1e-12):
        AL = self.A_cache[self.L]
        return compute_cost_and_dAL(AL, Y, eps)

Se definen las funciones de activación necesarias para cada uno de las funciones de la clase.

En este caso se recibe la topología y las activaciones por capa. Se inicializan los pesos de forma aleatoria y luego se comienza con cada uno de los métodos.

**Output**: Calcula el forward propagation y devuelve listas con Z y A de cada capa.

**Forward**: Realiza la propagación guardando en Z y A.

**Backward**: Calcula la derivada del costo respecto a AL, se aplica en cada capa y devuelve un diccionario.

Nuevamente se obtiene la función de costo para que la clase sea más completa.




In [4]:
def forward_pass(A0, nn_red):
    """
    A0: entrada inicial (X)
    nn_red: objeto de tipo NeuralNetwork

    Return:
      - salida final A
      - red actualizada (nn_red)
    """
    A = nn_red.forward(A0)
    return A, nn_red

Algunos ejemplos para comprobar el funcionamiento de la clase.

In [5]:
topology = [3, 4, 2, 1]
activations = [None, relu, relu, sigmoid]

nn = NeuralNetwork(topology, activations)

# Entrada inicial (3 características, 5 ejemplos)
A0 = np.random.randn(3, 5)

A_final, nn_actualizada = forward_pass(A0, nn)

print("Salida final:")
print(A_final)

print("\nDimensión Z[2]:", nn_actualizada.Z_cache[2].shape)
print("Dimensión A[2]:", nn_actualizada.A_cache[2].shape)

Salida final:
[[0.5034157  0.50341502 0.50341604 0.50341522 0.5034155 ]]

Dimensión Z[2]: (2, 5)
Dimensión A[2]: (2, 5)


In [6]:
# nn.A_cache[L] contiene AL con shape (1, m)
Y = np.array([[1, 0, 1, 1, 0]])
AL = nn.A_cache[nn.L]         # salida (1,5)

cost = nn.compute_cost(Y)
cost2, dAL = nn.compute_cost_and_dAL(Y)

print("Coste:", cost)
print("dAL shape:", dAL.shape)

Coste: 0.6918039169523441
dAL shape: (1, 5)


In [7]:
# 1. Forward pass
AL, nn = forward_pass(A0, nn)

# 2. Backward pass
grads = nn.backward(Y)

print("Gradiente de W2:", grads["dW2"])
print("Gradiente de b2:", grads["db2"])


Gradiente de W2: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.25291262e-05  4.11668539e-05 -2.82864099e-05 -1.18780094e-05]]
Gradiente de b2: [[0.        ]
 [0.00135036]]
